In [1]:
#  Install required packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5 langchain-community

In [2]:
!pip install -U langchain-community langchain-core

In [3]:
#  Imports
import os
from google.colab import userdata
import langchain, langchain_core
from langchain_google_genai import ChatGoogleGenerativeAI       # to work with Gemini models
from langchain_core.prompts import PromptTemplate               # to format prompts
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [4]:
#  Set API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")

In [11]:
#  1. Create Gemini LLM object
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7)

In [12]:
# 2. Define the Prompt (Uses MessagesPlaceholder for history)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"), # This replaces memory_key
    ("human", "{input}"),                             # This replaces input_key
])


In [13]:
# 3. Create the Chain (LCEL)
chain = prompt | llm

In [14]:
# 4. Setup Memory (State Management)
# In production, you'd swap ChatMessageHistory for Redis or SQL
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap the chain with history management
conversation_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [15]:
# 5. Run the conversation
name = input("Enter your name: ")
config = {"configurable": {"session_id": "user_session_1"}}

# First interaction
greet = conversation_chain.invoke(
    {"input": f"My name is {name}. Please remember it and greet me."},
    config=config
)
print("\nGreet:\n", greet.content)

print("\nAsking if it remembers your name...")

# Second interaction
response = conversation_chain.invoke(
    {"input": "Do you remember my name?"},
    config=config
)
print("\nResponse:\n", response.content)

Enter your name: Susan

Greet:
 Hello Susan! It's nice to meet you. I'll remember your name.

Asking if it remembers your name...

Response:
 Yes, I do! Your name is Susan.
